In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')

import plotly.offline as py 
py.init_notebook_mode(connected=True) # this code, allow us to work with offline plotly version
import plotly.graph_objs as go # it's like "plt" of matplot
#import plotly.tools as tls # It's useful to we get some tools of plotly

In [3]:
import folium
from folium import plugins
from folium import Map
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster

In [4]:
df = pd.read_csv("bases/questao1_coronavirus.csv", delimiter=";")

Usaremos como base de informação o site 
https://brasil.io/dataset/covid19/boletim/
que tem a siguinte documentação:

date: data de coleta dos dados no formato YYYY-MM-DD.

state: sigla da unidade federativa, exemplo: SP.

city: nome do município (pode estar em branco quando o registro é referente ao estado, pode ser preenchido com Importados/Indefinidos também).

new_confirmed: número de novos casos confirmados desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).

new_deaths: número de novos óbitos desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).

estimated_population_2019: população estimada para esse município/estado em 2019, segundo o IBGE (acesse o script que faz o download e conversão dos dados de população).

In [5]:
df.sample(10)

,date,state,city,new_confirmed,new_deaths,estimated_population_2019
22426,2020-06-17,PR,Catanduvas,3,0,10189.0
174173,2020-05-10,AL,Campo Alegre,0,0,57063.0
62423,2020-06-08,BA,Mortugaba,0,0,12042.0
8463,2020-06-20,RS,Capão do Leão,0,0,25354.0
176462,2020-05-10,SC,Penha,1,0,32531.0
140996,2020-05-21,SP,Luís Antônio,1,0,14947.0
106610,2020-05-29,CE,Viçosa do Ceará,0,1,60889.0
127157,2020-05-24,MA,São Francisco do Brejão,4,0,11798.0
96572,2020-06-01,SC,Ipuaçu,1,0,7514.0
156151,2020-05-16,MA,São Bernardo,0,0,28507.0


In [6]:
def estado_especifico(valor):
    estados = df.drop("city", 1)
    df_estado = estados.loc[estados.state==valor, :]
    df_estado = df_estado.groupby('date').sum()
    df_estado[['new_confirmed','new_deaths']] = df_estado[['new_confirmed','new_deaths']].cumsum()
    return df_estado

In [7]:
cidades = pd.read_csv("bases/cidades_brasil.csv")
cidades = cidades.rename(columns={"nome":"city"})

In [8]:
cidades.head()

,codigo_ibge,city,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


In [9]:
result = pd.merge(df, cidades, on='city')

In [10]:
result = result.dropna()

In [11]:
result.head()

,date,state,city,new_confirmed,new_deaths,estimated_population_2019,codigo_ibge,latitude,longitude,capital,codigo_uf
0,2020-06-21,AC,Acrelândia,0,0,15256.0,1200013,-9.82581,-66.8972,0,12
1,2020-06-20,AC,Acrelândia,0,0,15256.0,1200013,-9.82581,-66.8972,0,12
2,2020-06-19,AC,Acrelândia,0,0,15256.0,1200013,-9.82581,-66.8972,0,12
3,2020-06-18,AC,Acrelândia,1,0,15256.0,1200013,-9.82581,-66.8972,0,12
4,2020-06-17,AC,Acrelândia,2,0,15256.0,1200013,-9.82581,-66.8972,0,12


In [ ]:
coordenada = result[['latitude', 'longitude', 'new_confirmed']]

In [ ]:
coordenada = coordenada.dropna()

In [ ]:
basemap = folium.Map(
                    width="100%",
                    height="100%",
                    location= [-15.788497, -47.879873],
                    zoom_start=4
)

In [ ]:
marker_cluster = folium.MarkerCluster().add_to(basemap)

In [ ]:
for i in range(0, len(result)):
    folium.Marker(locationlist[result.iloc[i]['latitude'], result.iloc[i]['longitude']], 
                  popup=result['city'][result.iloc[i]['latitude'], result.iloc[i]['longitude']]
                 ).add_to(marker_cluster)


In [ ]:
basemap = basemap.add_child(plugins.HeatMap(coordenada))

In [ ]:
basemap

In [ ]:
for i in range(0, len(result)):
    folium.Circle(
        location= [result.iloc[i]['latitude'], result.iloc[i]['longitude']],
        color = '#00FF69',
        fill = '#00A1B3',
        tooltip = '<li><bold> Cidade: ' + str(result.iloc[i]['city']) +
                  '<li><bold> Estado: ' + str(result.iloc[i]['state']) +
                  '<li><bold> Casos : ' + str(result.iloc[i]['new_confirmed']) +
                  '<li><bold> Obitos: ' + str(result.iloc[i]['new_deaths']),
        radius = (result.iloc[i]['new_confirmed']**1.1)
    ).add_to(basemap)

In [ ]:
estado_sp = estado_especifico("SP")

In [ ]:
estado_ce = estado_especifico("CE")

In [ ]:
trace = [go.Scatter(x = estado_ce.index, y = estado_ce.new_confirmed),
        go.Scatter(x = estado_sp.index, y = estado_sp.new_confirmed)]

layout = dict(
    title='Ocorrências de Roubo de Celular',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='Último Mês',
                     step='month',
                     stepmode='backward'),
                dict(label='Tudo',
                     step='all'
                    )
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=trace, layout=layout)
py.iplot(fig)

In [ ]:
dist_df = df_estado.reset_index(level=[0,1])
dist_df[dist_df["state"]=="CE"].head()

In [ ]:
dist_df = df_estado.reset_index(level=[0,1])

In [ ]:
dist_df.head()

In [ ]:
lista_estado = dataframe_questao_1.state.unique()
populacao = 0
for i in lista_estado:
    print(df_estado[i].estimated_population_2019.max())

In [ ]:
def estado(valor):
    estados = dataframe_questao_1.drop("city", 1)
    df_estado = estados.loc[estados.state==valor, :]
    df_estado = df_estado.groupby('date').sum()
    df_estado[['new_confirmed','new_deaths']] = df_estado[['new_confirmed','new_deaths']].cumsum()
    return df_estado

In [ ]:
teste = estado("SP")
teste.head(10)

In [ ]:
teste =teste['2020-02-27' :'2020-03-02']

In [ ]:
teste.head(10)